In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchvision.models import ResNet50_Weights
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_dir = "/content/drive/MyDrive/Dataset"  
train_dir = f"{dataset_dir}/train"
val_dir = f"{dataset_dir}/val"
test_dir = f"{dataset_dir}/test"


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
transform = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}

In [ ]:
train_dataset = datasets.ImageFolder(train_dir, transform=transform["train"])
val_dataset = datasets.ImageFolder(val_dir, transform=transform["val"])
test_dataset = datasets.ImageFolder(test_dir, transform=transform["test"])


In [ ]:
batch_size = 128  # Optimized for A100(i was using Colab)
num_workers = 8  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, prefetch_factor=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, prefetch_factor=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, prefetch_factor=2)

In [ ]:
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 193MB/s]


In [ ]:
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scaler = torch.amp.GradScaler('cuda')


In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, scaler, epochs=10):
    best_val_acc = 0.0  # Track the best validation accuracy for model saving
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        print("-" * 20)

        # Training Phase
        model.train()
        train_loss, correct = 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad()

            # Mixed precision training
            with torch.amp.autocast('cuda'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)

        epoch_train_loss = train_loss / len(train_loader)
        epoch_train_acc = correct.double() / len(train_loader.dataset)
        print(f"Training Loss: {epoch_train_loss:.4f}, Accuracy: {epoch_train_acc:.4f}")

        # Validation Phase
        model.eval()
        val_loss, val_correct = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                with torch.amp.autocast('cuda'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        epoch_val_loss = val_loss / len(val_loader)
        epoch_val_acc = val_correct.double() / len(val_loader.dataset)
        print(f"Validation Loss: {epoch_val_loss:.4f}, Accuracy: {epoch_val_acc:.4f}")

        # Save the best model
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            torch.save(model.state_dict(), "catdog.pth")
            print(f"Model saved with validation accuracy: {best_val_acc:.4f}")


In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    test_correct = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            test_correct += torch.sum(preds == labels.data)

    test_accuracy = test_correct.double() / len(test_loader.dataset)
    print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
train_model(model, criterion, optimizer, train_loader, val_loader, scaler, epochs=10)



Epoch 1/10
--------------------
Training Loss: 0.0923, Accuracy: 0.9665
Validation Loss: 0.0643, Accuracy: 0.9752
Model saved with validation accuracy: 0.9752
Epoch 2/10
--------------------
Training Loss: 0.0493, Accuracy: 0.9822
Validation Loss: 0.1291, Accuracy: 0.9495
Epoch 3/10
--------------------
Training Loss: 0.0406, Accuracy: 0.9846
Validation Loss: 0.0718, Accuracy: 0.9695
Epoch 4/10
--------------------
Training Loss: 0.0372, Accuracy: 0.9861
Validation Loss: 0.1055, Accuracy: 0.9575
Epoch 5/10
--------------------
Training Loss: 0.0413, Accuracy: 0.9849
Validation Loss: 0.0611, Accuracy: 0.9780
Model saved with validation accuracy: 0.9780
Epoch 6/10
--------------------
Training Loss: 0.0270, Accuracy: 0.9902
Validation Loss: 0.0450, Accuracy: 0.9842
Model saved with validation accuracy: 0.9842
Epoch 7/10
--------------------
Training Loss: 0.0245, Accuracy: 0.9910
Validation Loss: 0.0885, Accuracy: 0.9645
Epoch 8/10
--------------------
Training Loss: 0.0277, Accuracy: 0.

In [ ]:
print("Evaluating the best model on test set...")
model.load_state_dict(torch.load("/content/catdog.pth"))
evaluate_model(model, test_loader)

Evaluating the best model on test set...


<ipython-input-14-2972772934d9>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/catdog.pth"))
/usr/local/lib/python3.10/dist-pack

Test Accuracy: 0.9858
